# Commands

In [ ]:
# cd ~/asa2-aa-logs ; jupython --no-tail --venv st-venv -o nohup-asa2-aa-$HOSTNAME.out ~/notebooks/asa/eval/asa2-aa.ipynb
# observe ~/asa2-aa-logs/nohup-asa2-aa-$HOSTNAME.out

# Init

In [ ]:
isNotebook = '__file__' not in locals()

In [ ]:
TEST = False # isNotebook, False, True

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Imports

In [ ]:
from newssource.asattribution.utils import *
from newssource.asattribution.asamin import *
from newssource.asa.asapreproc import *
from newssource.asa.models import *
from newssource.metrics.ndcg import *

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
if not isNotebook:
    matplotlib.use('Agg')
import random
import time
import pickle
import copy
from hashlib import md5

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Flatten
from keras.layers.embeddings import Embedding
from keras.models import load_model
from keras.utils import multi_gpu_model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
from random import random
from numpy import array
from numpy import cumsum
from keras.models import Sequential
from keras.layers import LSTM, GRU
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.utils import multi_gpu_model

In [ ]:
import statistics

In [ ]:
from machinelearning.baseline import *
from machinelearning.encoder import *
from machinelearning.kerasutils import *
from machinelearning.kerasmodels import *

In [ ]:
from machinelearning.baseline import *
from machinelearning.encoder import *
from machinelearning import kerasutils
from machinelearning.iterator import *
from machinelearning.metrics import *

In [ ]:
from keras.layers import concatenate, Input
from keras.models import Model
from keras.utils import plot_model

In [ ]:
import scipy

In [ ]:
from deepstyle.model import *

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
if not isNotebook:
    matplotlib.use('Agg')
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import linear_model

In [ ]:
# pip install Cython ; git clone https://github.com/epfml/sent2vec.git ; cd ./sent2vec ; pip install .

In [ ]:
from newssource.dbert.utils import *

In [ ]:
from nlptools.topicmodeling import *

# Config

In [ ]:
logger = Logger(tmpDir("logs") + "/asa2-aa-" + getHostname() + ".log")

In [ ]:
tt = TicToc(logger=logger)
tt.tic()

In [ ]:
files = \
[
    'uset0-l50-dpl50-d18-bc10',
    'uset0-l50-dpl50-blogger.com',
    'uset0-l50-dpl50-breitbart.com',
    'uset0-l50-dpl50-businessinsider.com',
    'uset0-l50-dpl50-cnn.com',
    'uset0-l50-dpl50-guardian.co.uk',
    'uset0-l50-dpl50-livejournal.com',
    'uset0-l50-dpl50-nytimes.com',
    'uset0-l50-dpl50-theguardian.com',
    'uset0-l50-dpl50-washingtonpost.com',
    'uset1-l50-dpl50-blogger.com',
    'uset1-l50-dpl50-d18-bc10',
    'uset1-l50-dpl50-livejournal.com',
    'uset2-l50-dpl50-blogger.com',
    'uset2-l50-dpl50-d18-bc10',
    'uset2-l50-dpl50-livejournal.com',
    'uset3-l50-dpl50-blogger.com',
    'uset3-l50-dpl50-d18-bc10',
    'uset3-l50-dpl50-livejournal.com',
    'uset4-l50-dpl50-blogger.com',
    'uset4-l50-dpl50-d18-bc10',
    'uset4-l50-dpl50-livejournal.com',
]
files = [nosaveDir() + "/Data/Asa2/detok-usets/" + e + "/0.ndjson.bz2" for e in files]
bp(files, 5, logger)

In [ ]:
tipisNumbers = "85 86 87 92 95 59 56 58 57 84 83 82 81 88 63 03 93 07 06 90 60 62 61 01 02 89 94".split()
tipis = ['tipi' + e for e in sorted(tipisNumbers)]
bp(tipis, logger)

In [ ]:
# We get the uset for the current tipi:
association = associate(tipis, files)
bp(association, 5, logger)
if not "tipi" in getHostname() or isDocker() or isHostname("titanv"):
    file = association['tipi07']
else:
    assert getHostname() in association
    file = association[getHostname()]
uset = file.split('/')[-2]

In [ ]:
# Parameters for results:
dataCol = "filtered_sentences" # filtered_sentences, sentences

In [ ]:
# Other parameters:
randomTestsAmount = 0
docLength = 1200

In [ ]:
# Logging:
log(getHostname() + " handles " + uset, logger)
log("dataCol: " + str(dataCol), logger)

In [ ]:
# Data:
hashs = []
docs = []
labels = []
flatDocs = []
flatLowDocs = []
flatTruncDocs = []
flatTruncLowDocs = []
detokDocs = []
detokSentences = []
for row in NDJson(file):
    detokSentences.append(row['filtered_detokenized_sentences'])
    detokDocs.append(row['filtered_detokenized'])
    sentences = row[dataCol]
    theHash = objectToHash(sentences)
    hashs.append(theHash)
    docs.append(sentences)
    labels.append(row["label"])
    flattenedDoc = flattenLists(sentences)
    flatDocs.append(flattenedDoc)
    flatLowDocs.append([e.lower() for e in flattenedDoc])
    truncatedDoc = flattenedDoc[:docLength]
    flatTruncDocs.append(truncatedDoc)
    flatTruncLowDocs.append([e.lower() for e in truncatedDoc])
bp(docs, logger)
tt.tic("Got documents")

In [ ]:
(classes, indexLabels) = encodeMulticlassLabels(labels, encoding='index')
bp(indexLabels, logger)

# Results

In [ ]:
(user, password, host) = getOctodsMongoAuth()

# Features:

In [ ]:
featuresCache = dict()
def getAsaFeatures2\
(
    docs,
    flatDocs,
    flatLowDocs,
    flatTruncDocs,
    flatTruncLowDocs,
    detokDocs,
    detokSentences,
    
    dataHash=None,
    
    useNMF=False,
    nmfKwargs=None,
    
    useLDA=False,
    ldaKwargs=None,
    
    useDbert=False,
    dbertKwargs=None, # operation, layer, modelName
    
    useDeepStyle=False,
    deepStyleKwargs=None,
    deepStyleRoot=nosaveDir() + "/asa2-train",
    
    useTFIDF=False,
    tfidfKwargs=None,
    defaultTFIDFNIter=30,
    
    useDoc2Vec=False,
    doc2VecKwargs=None,
    d2vPath=nosaveDir() + "/d2v/d2vmodel-t-ds22.02g-s300-w3-n15-e15-lTrue-adFalse-7bb8a",
    
    useStylo=False,
    styloKwargs=None,
    
    useUsent=False,
    usentKwargs=None,
    usentEmbeddingsPattern=nosaveDir() + "/usent/usentEmbedding*.pickle",
    
    useInferSent=False,
    inferSentKwargs=None,
    inferSentRoot=nosaveDir() + '/infersent',
    
    useBERT=False,
    bertKwargs=None,
    
    useSent2Vec=False,
    sent2VecKwargs=None,
    sent2VecRoot=nosaveDir() + '/sent2vec',
    defaultSent2VecModelName="wiki_unigrams.bin",

    logger=None,
    verbose=True,
):
    global featuresCache
    if dataHash is None:
        logWarning("Please provide a data hash to prevent its computation each call", logger)
        dataHash = objectToHash([docs, flatDocs, flatLowDocs, flatTruncDocs, flatTruncLowDocs, detokDocs, detokSentences])
    features = []
    # NMF:
    if useNMF:
        h = objectToHash(['NMF', nmfKwargs, dataHash])
        if h in featuresCache:
            features.append(featuresCache[h])
        else:
            data = nmfFeatures(flatDocs, **nmfKwargs)
            features.append(data)
            featuresCache[h] = data
    # LDA:
    if useLDA:
        h = objectToHash(['LDA', ldaKwargs, dataHash])
        if h in featuresCache:
            features.append(featuresCache[h])
        else:
            data = ldaFeatures(flatDocs, **ldaKwargs)
            features.append(data)
            featuresCache[h] = data
    # DBert:
    if useDbert:
        h = objectToHash(['DBert', dbertKwargs, dataHash])
        if h in featuresCache:
            features.append(featuresCache[h])
        else:
            modelName = dbertKwargs['modelName']
            if modelName is None:
                modelPath = None
            else:
                if isDir(nosaveDir() + '/dbert-train/' + modelName):
                    modelPath = sortedGlob(nosaveDir() + '/dbert-train/' + modelName + '/epochs/ep*')[-1]
                else:
                    modelPath = sortedGlob(nosaveDir() + '/dbert-tmp/' + modelName + '/epochs/ep*')[-1]
            layer = dbertKwargs['layer']
            # if modelName is None:
            #     (user, password, host) = getOctodsMongoAuth()
            #     dbertCache = SerializableDict("dbert-embeddings",
            #                                   user=user, host=host, password=password,
            #                                   useMongodb=True, logger=logger)
            # else:
            dbertCache = SerializableDict("dbert-embeddings-" + str(modelName),
                          nosaveDir() + '/dbert-cache',
                          useMongodb=False, logger=logger,
                          loadRetry=30, loadSleepMin=0.5, loadSleepMax=30,
                         readIsAnAction=False,)
            embeddings = []
            for doc in docs:
                if modelPath is None:
                    layer = 'distilbert'
                (currentHash, cacheObject) = getDbertEmbeddingsHash(doc, layer, modelPath)
                if modelPath is None:
                    layer = None
                assert currentHash in dbertCache
                embeddings.append(dbertCache[currentHash]['embeddings'])
            dbertData = []
            if dbertKwargs['operation'] == "first":
                for emb in embeddings:
                    dbertData.append(emb[0])
            elif dbertKwargs['operation'] == "mean":
                for emb in embeddings:
                    dbertData.append(np.mean(emb, axis=0))
            dbertData = np.array(dbertData)
            features.append(dbertData)
            featuresCache[h] = dbertData
    # DeepStyle:
    if useDeepStyle:
        h = objectToHash(['DeepStyle', deepStyleKwargs, dataHash])
        if h in featuresCache:
            features.append(featuresCache[h])
        else:
            m = DeepStyle(deepStyleRoot + "/" + deepStyleKwargs['modelPattern'])
            embeddings = np.array([m.embed(doc) for doc in docs])
            features.append(embeddings)
            featuresCache[h] = embeddings
    # TFIDF:
    if useTFIDF:
        if "nIter" not in tfidfKwargs:
            tfidfKwargs["nIter"] = defaultTFIDFNIter
        h = objectToHash(['TFIDF', tfidfKwargs, dataHash])
        if h in featuresCache:
            features.append(featuresCache[h])
        else:
            data = flatTruncDocs if tfidfKwargs['truncate'] else docs
            tfidfInstance = TFIDF(data, doLower=tfidfKwargs['doLower'], logger=logger, verbose=False)
            tfidfData = tfidfInstance.getTFIDFMatrix()
            svd = TruncatedSVD(n_components=tfidfKwargs['nComponents'],
                               n_iter=tfidfKwargs['nIter'],
                               random_state=42)
            svdTFIDFData = svd.fit_transform(tfidfData)
            features.append(svdTFIDFData)
            featuresCache[h] = svdTFIDFData
    # Doc2Vec:
    if useDoc2Vec:
        h = objectToHash(['Doc2Vec', doc2VecKwargs, dataHash])
        if h in featuresCache:
            features.append(featuresCache[h])
        else:
            data = flatTruncLowDocs if doc2VecKwargs['truncate'] else flatLowDocs
            d2vModel = Doc2Vec.load(sortedGlob(d2vPath + "/*model*.d2v")[0])
            d2vData = d2vTokenssToEmbeddings(data, d2vModel, logger=logger, verbose=False)
            features.append(d2vData)
            featuresCache[h] = d2vData
    # Stylo:
    if useStylo:
        h = objectToHash(['Stylo', styloKwargs, dataHash])
        if h in featuresCache:
            features.append(featuresCache[h])
        else:
            styloVectors = []
            for text in pb(detokDocs, logger=logger, message="Getting stylo features", verbose=verbose):
                styloVectors.append(stylo(text, asNpArray=True))
            styloVectors = np.array(styloVectors)
            features.append(styloVectors)
            featuresCache[h] = styloVectors
    # Usent:
    if useUsent:
        h = objectToHash(['Usent', usentKwargs, dataHash])
        if h in featuresCache:
            features.append(featuresCache[h])
        else:
            # We get all embeddgins from usent:
            allHashes = set()
            for doc in docs:
                docHash = objectToHash(doc)
                allHashes.add(docHash)
                for sentence in doc:
                    theHash = objectToHash(sentence)
                    allHashes.add(theHash)
            usentEmbeddings = dict()
            for usentEmbeddingsFile in pb(sortedGlob(usentEmbeddingsPattern),
                                          printRatio=0.1, logger=logger, message="Getting Usent embeddings from all files"):
                current = deserialize(usentEmbeddingsFile)
                for theHash, value in current.items():
                    if theHash in allHashes:
                        usentEmbeddings[theHash] = value
            assert len(allHashes) == len(usentEmbeddings)
            if usentKwargs['operation'] == "full":
                data = []
                for doc in docs:
                    theHash = objectToHash(doc)
                    data.append(usentEmbeddings[theHash])
                data = np.array(data)
            elif usentKwargs['operation'] == "mean":
                data = []
                for doc in docs:
                    docEmbeddings = []
                    for sentence in doc:
                        theHash = objectToHash(sentence)
                        docEmbeddings.append(usentEmbeddings[theHash])
                    docEmbedding = np.mean(docEmbeddings, axis=0)
                    data.append(docEmbedding)
                data = np.array(data)
            features.append(data)
            featuresCache[h] = data
    # InferSent:
    if useInferSent:
        h = objectToHash(['InferSent', inferSentKwargs, dataHash])
        if h in featuresCache:
            features.append(featuresCache[h])
        else:
            V = inferSentKwargs['V']
            operation = inferSentKwargs['operation']
            MODEL_PATH = inferSentRoot + '/infersent%s.pkl' % V
            params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                            'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
            infersent = InferSent(params_model)
            infersent.load_state_dict(torch.load(MODEL_PATH))
            if V == 2:
                W2V_PATH = inferSentRoot + '/fastText/crawl-300d-2M.vec'
            else:
                W2V_PATH = inferSentRoot + '/GloVe/glove.840B.300d.txt'
            infersent.set_w2v_path(W2V_PATH)
            infersent.build_vocab(detokDocs, tokenize=True)
            if operation == "full":
                detokDocsForInferSent = []
                for current in detokDocs:
                    detokDocsForInferSent.append(current[:10000])
                isData = infersent.encode(detokDocsForInferSent, tokenize=True)
            elif operation == "mean":
                isData = []
                for currentDetokSentences in detokSentences:
                    embedding = np.mean(infersent.encode(currentDetokSentences, tokenize=True), axis=0)
                    isData.append(embedding)
                isData = np.array(isData)
            features.append(isData)
            featuresCache[h] = isData
    # BERT:
    if useBERT:
        h = objectToHash(['BERT', bertKwargs, dataHash])
        if h in featuresCache:
            features.append(featuresCache[h])
        else:
            (user, password, host) = getOctodsMongoAuth()
            bertCache = SerializableDict("newsid-bretcache",
                                         useMongodb=True,
                                         user=user, password=password, host=host,
                                         logger=logger)
            bertData = []
            for doc in docs:
                theHash = objectToHash(doc)
                current = bertCache[theHash]
                bertData.append(current)
            bertData = np.array(bertData)
            features.append(bertData)
            featuresCache[h] = bertData
    # Sent2Vec:
    if useSent2Vec:
        if sent2VecKwargs is None:
            sent2VecKwargs = dict()
        if 'modelName' not in sent2VecKwargs:
            sent2VecKwargs['modelName'] = defaultSent2VecModelName
        h = objectToHash(['Sent2Vec', sent2VecKwargs, dataHash])
        if h in featuresCache:
            features.append(featuresCache[h])
        else:
            modelName = sent2VecKwargs['modelName']
            s2vModel = sent2vec.Sent2vecModel()
            s2vModel.load_model(sent2VecRoot + '/' + modelName)
            operation = sent2VecKwargs['operation']
            if operation == "full":
                s2vData = s2vModel.embed_sentences(detokDocs)
            elif operation == "mean":
                s2vData = []
                for currentDetokSentences in detokSentences:
                    embedding = np.mean(s2vModel.embed_sentences(currentDetokSentences), axis=0)
                    s2vData.append(embedding)
                s2vData = np.array(s2vData)
            try:
                s2vModel.release_shared_mem(sent2VecRoot + '/' + modelName)
                s2vModel = None
            except Exception as e:
                logException(e, logger)
            features.append(s2vData)
            featuresCache[h] = s2vData
    # Concatenation:
    features = np.concatenate(features, axis=1)
    return features

# Execution

In [ ]:
def getDoubleCombinasons():
    for i in range(len(uniqueUses)):
        for u in range(i + 1, len(uniqueUses)):
            yield (uniqueUses[i], uniqueUses[u])

In [ ]:
def getScore(features, labels, validationRatio=0.3, doFigShow=isNotebook, logger=None, verbose=True):
    clf = linear_model.SGDClassifier()
    xTrain, xTest, yTrain, yTest = train_test_split(features, labels, test_size=validationRatio, random_state=42)
    scores = scikitLearnFit(clf, xTrain, yTrain, xTest, yTest, doFigShow=doFigShow, doFigSave=False, logger=logger)
    log("Features shape: " + str(features.shape), logger=logger, verbose=verbose)
    return max(scores)

In [ ]:
uniqueUses = ['useNMF', 'useLDA', 'useDbert', 'useTFIDF', 'useStylo', 'useDeepStyle', 'useBERT', 'useDoc2Vec', 'useUsent', 'useInferSent', 'useSent2Vec']
allKwargs = \
{
    'nmfKwargs': {},
    'ldaKwargs': {},
    # 'dbertKwargs': {'operation': 'mean', 'layer': 'distilbert', 'modelName': '94bef_ep32'},
    'dbertKwargs': {'operation': 'mean', 'layer': None, 'modelName': None},
    'deepStyleKwargs': {'modelPattern': '6ebdd3e05d4388c658ca2d5c53b0bc36'},
    'tfidfKwargs': {'truncate': False, 'doLower': True, 'nComponents': 50},
    'doc2VecKwargs': {'truncate': False},
    'styloKwargs': None,
    'usentKwargs': {'operation': 'mean'},
    'inferSentKwargs': {'V': 1, 'operation': 'mean'},
    'bertKwargs': None,
    'sent2VecKwargs': {'operation': 'mean'},
}
useToKwargsMap = \
{
    'useNMF': 'nmfKwargs',
    'useLDA': 'ldaKwargs',
    'useDbert': 'dbertKwargs',
    'useDeepStyle': 'deepStyleKwargs',
    'useTFIDF': 'tfidfKwargs',
    'useDoc2Vec': 'doc2VecKwargs',
    'useStylo': 'styloKwargs',
    'useUsent': 'usentKwargs',
    'useInferSent': 'inferSentKwargs',
    'useBERT': 'bertKwargs',
    'useSent2Vec': 'sent2VecKwargs',
}

In [ ]:
combinasons = [(e,) for e in uniqueUses] + list(getDoubleCombinasons())
bp(combinasons, 5, logger)
log("Count of combinasons: " + str(len(combinasons)), logger)
log("Count of points: " + str(len(combinasons) * len(files)), logger)

In [ ]:
if True:
    logWarning("Removing combs that do not have useDbert", logger)
    newCombinasons = []
    for current in combinasons:
        # if 'useDbert' in current and ('useNMF' in current or 'useLDA' in current):
        # if 'useNMF' in current or 'useLDA' in current:
        if 'useDbert' in current:
        # if 'useDbert' in current and 'useNMF' in current:
            newCombinasons.append(current)
    combinasons = newCombinasons
    bp(combinasons, 3, logger)

In [ ]:
if isNotebook:
    combinasons = combinasons[:1]
    bp(combinasons, 3, logger)

In [ ]:
documentsArgs = \
(
    docs,
    flatDocs,
    flatLowDocs,
    flatTruncDocs,
    flatTruncLowDocs,
    detokDocs,
    detokSentences,
)

In [ ]:
dataHash = objectToHash(documentsArgs)
bp('dataHash: ' + dataHash, logger)

In [ ]:
# AA:
if True:
    results = SerializableDict('asa2-aa', useMongodb=True, user=user, password=password, host=host, logger=logger)
    for comb in pb(combinasons, logger=logger):
        currentKwargs = copy.deepcopy(allKwargs)
        for unique in comb:
            currentKwargs[unique] = True
        result = copy.deepcopy(currentKwargs)
        for key in useToKwargsMap:
            if not (dictContains(result, key) and result[key]):
                del result[useToKwargsMap[key]]
        result['uset'] = uset
        result['dataCol'] = dataCol
        theHash = objectToHash(result)
        if theHash not in results:
            features = getAsaFeatures2\
            (
                *documentsArgs,
                logger=logger,
                verbose=True,
                dataHash=dataHash,
                **currentKwargs,
            )
            score = getScore(features, indexLabels, logger=logger, doFigShow=False)
            log("Score of " + str(comb) + ": " + str(truncateFloat(score, 3)), logger)
            result['score'] = score
            results[theHash] = result
        else:
            log("Found the score of " + str(comb) + ": " + str(truncateFloat(results[theHash]['score'], 3)), logger)

In [ ]:
# Clustering:
if False:
    results = SerializableDict('asa2-clustering-comb', useMongodb=True, user=user, password=password, host=host, logger=logger)
    for comb in pb(combinasons, logger=logger):
        currentKwargs = copy.deepcopy(allKwargs)
        for unique in comb:
            currentKwargs[unique] = True
        result = copy.deepcopy(currentKwargs)
        for key in useToKwargsMap:
            if not (dictContains(result, key) and result[key]):
                del result[useToKwargsMap[key]]
        result['uset'] = uset
        result['dataCol'] = dataCol
        theHash = objectToHash(result)
        if theHash not in results:
            features = getAsaFeatures2\
            (
                *documentsArgs,
                logger=logger,
                verbose=True,
                dataHash=dataHash,
                **currentKwargs,
            )
            # We compute and store score:
            data = features
            simMatrix = pairwiseCosineSimilarity(data)
            score = pairwiseSimNDCG(simMatrix, indexLabels)
            log("SimRank of " + str(comb) + ": " + str(truncateFloat(score, 3)), logger)
            # calharScore = metrics.calinski_harabasz_score(data, indexLabels)
            # log("CalHar: " + str(calharScore), logger)
            # davbScore = metrics.davies_bouldin_score(data, indexLabels)
            # log("DavB: " + str(davbScore), logger)
            # Adding results:
            result['score'] = score
            results[theHash] = result
        else:
            log("Found the score of " + str(comb) + ": " + str(truncateFloat(results[theHash]['score'], 3)), logger)

In [ ]:
if False:
    allKwargs['dbertKwargs'] = {'operation': 'mean', 'layer': None, 'modelName': None}
    currentKwargs = copy.deepcopy(allKwargs)
    currentKwargs['useDbert'] = True
    dbertBaseFeatures = getAsaFeatures2\
    (
        *documentsArgs,
        logger=logger,
        verbose=True,
        dataHash=dataHash,
        **currentKwargs,
    )
    bp(dbertBaseFeatures, logger)
    allKwargs['dbertKwargs'] = {'operation': 'mean', 'layer': 'distilbert', 'modelName': '94bef_ep32'}
    currentKwargs = copy.deepcopy(allKwargs)
    currentKwargs['useDbert'] = True
    dbertEp32Features = getAsaFeatures2\
    (
        *documentsArgs,
        logger=logger,
        verbose=True,
        dataHash=dataHash,
        **currentKwargs,
    )
    bp(dbertEp32Features, logger)

In [ ]:
if False:
    features = np.concatenate([dbertEp32Features, dbertBaseFeatures], axis=1)
    bp(dbertEp32Features, logger)
    log(features.shape, logger)
    score = getScore(features, indexLabels, logger=logger, doFigShow=False)
    log("Score of bdert base and 94bef_ep32 for " + uset + " --> " + str(score), logger)

In [ ]:
if False:
    features = np.concatenate([dbertEp32Features, dbertBaseFeatures], axis=1)
    data = features
    simMatrix = pairwiseCosineSimilarity(data)
    score = pairwiseSimNDCG(simMatrix, indexLabels)
    log("simrank_uuu Score of bdert base and 94bef_ep32 for " + uset + " --> " + str(score), logger)

# End

In [ ]:
tt.toc()

In [ ]:
if not isNotebook:
    exit()

# Tables

### Uniques

In [ ]:
results = SerializableDict('asa2-aa', useMongodb=True, user=user, password=password, host=host, logger=logger)
# results = SerializableDict('asa2-clustering-comb', useMongodb=True, user=user, password=password, host=host, logger=logger)

In [ ]:
def extractFeatures(o):
    k = set(o.keys()) if isinstance(o, dict) else set(o)
    return [k for k in o if re.search("^use[A-Z].*$", k)]
def featuresCount(*args, **kwargs):
    return len(extractFeatures(*args, **kwargs))

In [ ]:
def getData(fCount=None, usetPattern=None):
    if usetPattern is None:
        usetPattern = '.*'
    if ".*" not in usetPattern:
        usetPattern = ".*" + usetPattern + ".*"
    if '^' not in usetPattern:
        usetPattern = '^' + usetPattern
    if '$' not in usetPattern:
        usetPattern = usetPattern + '$'
    r = []
    for _, e in results.items():
        if re.match(usetPattern, e['uset']) and (fCount is None or featuresCount(e) == fCount):
            r.append(e)
    return r

In [ ]:
# r = getData(1, uset)
# r = getData(1, 'd18-bc10')
# r = getData(1, '.*')
# r = getData(1, 'blogger')
# r = getData(1, 'livejournal')
r = getData(1, 'washington')
# r = getData(1, 'breitbart')
# r = getData(1, 'business')
# r = getData(1, 'cnn')
# r = getData(1, 'guardian.co.uk')
# r = getData(1, 'theguardian.com')
# r = getData(1, 'nytimes')

In [ ]:
def filterDbert(r, dbertKwargsFilter, logger=None, verbose=True):
    logWarning("Filtering DBert with " + str(dbertKwargsFilter), logger)
    deletedCount = 0
    acceptedCount = 0
    newR = []
    for current in r:
        if dictContains(current, 'useDbert') and current['useDbert']:
            ok = True
            for key in dbertKwargsFilter:
                if current['dbertKwargs'][key] != dbertKwargsFilter[key]:
                    ok = False
                    break
            if ok:
                newR.append(current)
                acceptedCount += 1
            else:
                deletedCount += 1
        else:
            newR.append(current)
    log("deletedCount: " + str(deletedCount), logger)
    log("acceptedCount: " + str(acceptedCount), logger)
    return newR

In [ ]:
# r = filterDbert(r, {'operation': 'mean', 'layer': None, 'modelName': None}, logger=logger)
r = filterDbert(r, {'operation': 'mean', 'layer': 'distilbert', 'modelName': '94bef_ep32'}, logger=logger)

In [ ]:
bp(r, 5, logger)

In [ ]:
models = dict()

In [ ]:
log("Amount of usets: " + str(len(set([e['uset'] for e in r]))))
log("Amount of models: " + str(len(set([str(extractFeatures(e)) for e in r]))))

In [ ]:
for current in r:
    f = extractFeatures(current)[0]
    if f not in models:
        models[f] = []
    models[f].append(current['score'])
bp(models, 5)

In [ ]:
for key, values in models.items():
    print(len(values))

In [ ]:
for key in models.keys():
    models[key] = float(np.mean(models[key]))
for m, s in sortBy(models.items(), 1): print(m, s)

### Doubles

In [ ]:
# results = SerializableDict('asa2-aa', useMongodb=True, user=user, password=password, host=host, logger=logger)
results = SerializableDict('asa2-clustering-comb', useMongodb=True, user=user, password=password, host=host, logger=logger)

In [ ]:
r = getData(2, '.*')
# r = getData(2, 'd18-bc10')
# r = getData(2, 'blogger')
# r = getData(2, 'livejournal')
# r = getData(2, 'washington')
# r = getData(2, 'breitbart')
# r = getData(2, 'business')
# r = getData(2, 'cnn')
# r = getData(2, 'guardian.co.uk')
# r = getData(2, 'theguardian.com')
# r = getData(2, 'nytimes')

In [ ]:
r = filterDbert(r, {'operation': 'mean', 'layer': None, 'modelName': None}, logger=logger)
# r = filterDbert(r, {'operation': 'mean', 'layer': 'distilbert', 'modelName': '94bef_ep32'}, logger=logger)

In [ ]:
log("Amount of usets: " + str(len(set([e['uset'] for e in r]))))
log("Amount of models: " + str(len(set([str(extractFeatures(e)) for e in r]))))

In [ ]:
models = dict()

In [ ]:
for current in r:
    f = str(extractFeatures(current))
    if f not in models:
        models[f] = []
    models[f].append(current['score'])
bp(models)

In [ ]:
for e in sorted(models.keys()):
    print(e)

In [ ]:
for key, values in models.items():
    print(len(values))

In [ ]:
for key in models.keys():
    models[key] = float(np.mean(models[key]))

In [ ]:
for m, s in sortBy(models.items(), 1):
    print(m, s)